In [ ]:
import zarr
import numpy as np
import json
from main import main

'''
Run inference on a small sample image from TCGA
'''
params = {

}

main(params)

In [ ]:
'''
Instance map: 2D full-size matrix where each pixels value corresponds to the associated instance (value>0) or background (value=0)
'''

# open: file-like interaction with zarr-array
instance_map = zarr.open("pinst_pp.zip", mode="r")
# selecting a ROI will yield a numpy array
roi = instance_map[10000:20000,10000:20000]
# or with [:] to load the entire array
full_instance_map = instance_map[:]
# alternatively, use load, which will directly create a numpy array:
full_instance_map = zarr.load("pinst_pp.zip") 

'''
Class dictionary: Lookup for the instance map, also contains centroid coordinates. If only centroid coordinates are of interest, you can skip loading the instance map.
'''

# load the dictionary
with open("class_inst.json","r") as f:
    class_info = json.load(f)
# create a centroid info array
centroid_array = np.array([[int(k),v[0],*v[1]] for k,v in class_info.items()])
# [instance_id, class_id, y, x]

# or alternatively create a lookup for the instance map to get a corresponding class map
pcls_list = np.array([0] + [v[0] for v in class_info.values()])
pcls_keys = np.array(["0"] + list(class_info.keys())).astype(int)
lookup = np.zeros(pcls_keys.max() + 1,dtype=np.uint8)
lookup[pcls_keys] = pcls_list
cls_map = lookup[full_instance_map]